# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 24, 10, 25

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████| 20/20 [02:57<00:00,  8.87s/it]


,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
9,PriceChange,10,91.30,78.26,78.26,73.91,78.26,69.57,69.57,69.57,65.22,73.91,78.26,65.22,65.22,65.22,73.91,69.57,69.57,78.26,78.26,73.91,78.26,78.26,78.26,82.61,-1.50,-1.20,-0.94,-0.90,-0.74,-1.20,-0.72,-0.82,-0.89,-0.95,-1.49,-1.34,-0.58,-0.80,-0.80,-0.95,-0.90,-1.16,-1.33,-1.33,-1.41,-1.74,-2.63,-2.56,23,74.275833,98.344167
10,PriceChange,11,92.00,76.00,76.00,72.00,76.00,72.00,72.00,72.00,68.00,76.00,80.00,64.00,64.00,64.00,72.00,64.00,64.00,72.00,72.00,72.00,76.00,76.00,76.00,84.00,-1.50,-1.20,-0.94,-0.90,-1.27,-1.20,-0.94,-1.39,-1.00,-0.96,-1.49,-1.37,-0.90,-1.19,-1.13,-0.95,-0.90,-1.16,-1.19,-1.19,-1.19,-1.74,-2.63,-2.56,25,73.000000,75.000000
1,PriceChange,2,100.00,33.33,66.67,100.00,100.00,66.67,66.67,66.67,66.67,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-0.53,0.25,-6.35,-8.10,-7.37,-8.01,-7.42,-7.07,-6.88,-7.22,-6.09,-10.85,-9.18,-7.96,-7.96,-7.91,-10.07,-10.27,-10.61,-9.23,-9.23,-10.17,-10.07,-8.99,3,90.278333,60.835000
0,PriceChange,1,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-6.55,-10.14,-12.19,-12.47,-12.84,-12.67,-12.87,-12.69,-12.60,-12.77,-12.70,-15.08,-13.75,-12.64,-12.15,-12.37,-13.70,-13.80,-13.97,-13.53,-13.53,-13.25,-12.46,-13.72,2,100.000000,60.000000
15,PriceChange,16,94.59,59.46,72.97,64.86,62.16,62.16,64.86,64.86,67.57,70.27,75.68,62.16,67.57,64.86,70.27,67.57,67.57,72.97,72.97,75.68,78.38,78.38,78.38,86.49,-0.81,-0.33,-0.59,-0.54,-0.48,-0.76,-0.60,-0.44,-0.58,-0.82,-1.00,-0.98,-0.96,-1.19,-1.19,-1.04,-0.91,-1.06,-1.19,-1.58,-1.66,-1.70,-1.83,-2.10,37,70.945417,34.980417
8,PriceChange,9,88.89,72.22,77.78,72.22,72.22,61.11,61.11,61.11,55.56,66.67,72.22,61.11,66.67,61.11,72.22,66.67,66.67,77.78,77.78,72.22,77.78,77.78,77.78,77.78,-1.85,-1.28,-1.06,-1.01,-1.00,-1.28,-0.91,-0.70,-0.52,-0.88,-0.80,-1.12,-0.56,-0.72,-0.70,-0.64,-0.64,-0.88,-1.18,-1.26,-1.30,-1.50,-1.76,-1.70,18,70.602500,10.845000
11,PriceChange,12,92.86,67.86,78.57,71.43,71.43,67.86,67.86,71.43,67.86,71.43,78.57,60.71,64.29,60.71,67.86,60.71,60.71,67.86,67.86,67.86,71.43,71.43,71.43,82.14,-1.48,-1.08,-0.91,-0.86,-0.67,-0.97,-0.71,-0.56,-0.72,-0.92,-1.16,-1.27,-0.74,-0.92,-0.87,-0.59,-0.49,-0.88,-0.99,-1.01,-1.14,-1.30,-1.72,-1.60,28,70.090000,2.520000
14,PriceChange,15,94.29,57.14,71.43,62.86,60.00,60.00,62.86,62.86,65.71,68.57,74.29,60.00,65.71,62.86,68.57,65.71,65.71,71.43,71.43,74.29,77.14,77.14,77.14,85.71,-0.81,-0.24,-0.58,-0.51,-0.47,-0.76,-0.51,-0.43,-0.56,-0.77,-0.71,-0.90,-0.94,-0.92,-1.16,-1.01,-0.90,-0.94,-1.16,-1.56,-1.65,-1.56,-1.80,-2.03,35,69.285417,-25.010417
18,PriceChange,19,90.00,60.00,68.00,64.00,64.00,62.00,62.00,62.00,62.00,66.00,68.00,66.00,70.00,68.00,70.00,70.00,66.00,72.00,70.00,74.00,76.00,76.00,76.00,82.00,-0.55,-0.26,-0.58,-0.38,-0.47,-0.49,-0.54,-0.51,-0.33,-0.59,-0.58,-0.62,-0.91,-0.90,-1.14,-0.88,-0.86,-0.89,-1

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [10]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)